In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from math import sqrt
import matplotlib.pyplot as plt

# Load the CSV data file
data = pd.read_csv('/content/rssi.csv')

# Check for NaN or infinite values in your data
print(data.isna().sum())
print(data.isin([np.nan, np.inf, -np.inf]).sum())

# Handle missing or infinite values (if any)
data.dropna(inplace=True)
data.replace([np.inf, -np.inf], np.nan, inplace=True)
data.dropna(inplace=True)

# Split data into features (RSSI) and labels (x, y, z)
X = data[['signal', 'ap']]
y = data[['x', 'y', 'z']]

# Convert 'ap' column into one-hot encoding
X = pd.get_dummies(X, columns=['ap'])

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Reshape the input data to include time steps (1 in this case)
X = X.reshape(X.shape[0], 1, X.shape[1])

# Split the data into training (70%) and testing (30%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Build the LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(LSTM(64))
model.add(Dense(3))  # 3 output neurons for x, y, and z

# Add dropout layers for regularization
model.add(Dropout(0.2))
model.add(Dropout(0.2))

# Compile the model with a lower learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Train the model with more epochs and a smaller batch size
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on the testing data
y_pred = model.predict(X_test)

# Calculate the root mean squared error (RMSE)
rmse = sqrt(mean_squared_error(y_test, y_pred))
print(f'RMSE: {rmse}')

# Inverse transform predictions to the original scale
y_pred = scaler.inverse_transform(y_pred)  # Inverse transform predictions to the original scale
y_true = scaler.inverse_transform(y_test)   # Inverse transform actual values to the original scale

plt.figure(figsize=(12, 6))
plt.plot(y_true, label='Actual', color='blue')
plt.plot(y_pred, label='Predicted', color='red')
plt.title('Actual vs. Predicted Location')
plt.xlabel('Sample')
plt.ylabel('Location (x, y, z)')
plt.legend()
plt.show()

# Optionally, save the trained model
model.save('location_prediction_model.h5')





ap          1
signal      1
sequence    1
x           1
y           1
z           1
dtype: int64
ap          1
signal      1
sequence    1
x           1
y           1
z           1
dtype: int64
Epoch 1/100
2625/2625 [==============================] - 25s 8ms/step - loss: 142.9108 - val_loss: 83.3905
Epoch 2/100
2625/2625 [==============================] - 19s 7ms/step - loss: 136.3084 - val_loss: 84.8832
Epoch 3/100
2625/2625 [==============================] - 20s 7ms/step - loss: 134.4453 - val_loss: 84.6812
Epoch 4/100
2625/2625 [==============================] - 21s 8ms/step - loss: 136.0146 - val_loss: 83.4575
Epoch 5/100
2625/2625 [==============================] - 21s 8ms/step - loss: 135.1787 - val_loss: 83.8509
Epoch 6/100
2625/2625 [==============================] - 19s 7ms/step - loss: 135.1736 - val_loss: 81.9492
Epoch 7/100
2625/2625 [==============================] - 19s 7ms/step - loss: 135.4615 - val_loss: 80.7590
Epoch 8/100
2625/2625 [==============================] - 

ValueError: ignored

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout
from math import sqrt
import matplotlib.pyplot as plt

# Load the CSV data file
data = pd.read_csv('/content/rssi.csv')

# Handle missing or infinite values (if any)
data.dropna(inplace=True)
data.replace([np.inf, -np.inf], np.nan, inplace=True)
data.dropna(inplace=True)

# Split data into features (RSSI) and labels (x, y, z)
X = data[['signal', 'ap']]
y = data[['x', 'y', 'z']]

# Separate numeric and categorical columns
numeric_features = ['signal']
categorical_features = ['ap']

# Create transformers for numeric and categorical features
numeric_transformer = Pipeline(steps=[
    ('scaler', MinMaxScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(drop='first'))  # Drop first category to avoid multicollinearity
])

# Use ColumnTransformer to apply different transformers to different columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Define the complete model pipeline
model = Sequential()
model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=(X_train_preprocessed.shape[1], X_train_preprocessed.shape[2])))
model.add(LSTM(64))
model.add(Dense(3))  # 3 output neurons for x, y, and z
model.add(Dropout(0.2))
model.add(Dropout(0.2))
model.compile(optimizer='adam', loss='mean_squared_error')

# Split the data into training (70%) and testing (30%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Apply the preprocessing pipeline to the features
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

# Reshape the input data to include time steps (1 in this case)
X_train_preprocessed = X_train_preprocessed.reshape(X_train_preprocessed.shape[0], 1, X_train_preprocessed.shape[1])
X_test_preprocessed = X_test_preprocessed.reshape(X_test_preprocessed.shape[0], 1, X_test_preprocessed.shape[1])

# Train the model with more epochs and a smaller batch size
model.fit(X_train_preprocessed, y_train, epochs=200, batch_size=32, validation_data=(X_test_preprocessed, y_test), verbose=2)

# Evaluate the model on the testing data
y_pred = model.predict(X_test_preprocessed)

# Calculate the root mean squared error (RMSE)
rmse = sqrt(mean_squared_error(y_test, y_pred))
print(f'RMSE: {rmse}')

# Optionally, save the trained model
model.save('location_prediction_model.h5')

# Plot actual vs predicted values (no inverse scaling)
plt.figure(figsize=(12, 6))
plt.plot(y_test.values, label='Actual', color='blue')
plt.plot(y_pred, label='Predicted', color='red')
plt.title('Actual vs. Predicted Location')
plt.xlabel('Sample')
plt.ylabel('Location (x, y, z)')
plt.legend()
plt.show()


Epoch 1/200
2625/2625 - 34s - loss: 151.7586 - val_loss: 100.8401 - 34s/epoch - 13ms/step
Epoch 2/200
2625/2625 - 26s - loss: 146.8359 - val_loss: 96.7488 - 26s/epoch - 10ms/step
Epoch 3/200
2625/2625 - 27s - loss: 142.1207 - val_loss: 92.8496 - 27s/epoch - 10ms/step
Epoch 4/200
2625/2625 - 25s - loss: 139.7017 - val_loss: 87.3896 - 25s/epoch - 10ms/step
Epoch 5/200
2625/2625 - 27s - loss: 137.5450 - val_loss: 87.1524 - 27s/epoch - 10ms/step
Epoch 6/200
2625/2625 - 28s - loss: 137.0067 - val_loss: 84.3254 - 28s/epoch - 11ms/step
Epoch 7/200
2625/2625 - 25s - loss: 136.4916 - val_loss: 87.2172 - 25s/epoch - 10ms/step
Epoch 8/200
2625/2625 - 26s - loss: 135.7164 - val_loss: 84.2469 - 26s/epoch - 10ms/step
Epoch 9/200
2625/2625 - 28s - loss: 134.7743 - val_loss: 84.6935 - 28s/epoch - 11ms/step
Epoch 10/200
2625/2625 - 25s - loss: 135.2610 - val_loss: 84.4790 - 25s/epoch - 10ms/step
Epoch 11/200
2625/2625 - 24s - loss: 134.3960 - val_loss: 82.8505 - 24s/epoch - 9ms/step
Epoch 12/200
2625/2

KeyboardInterrupt: ignored

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.optimizers import Adam
from math import sqrt
import matplotlib.pyplot as plt

# Load your CSV data file for location prediction
data = pd.read_csv('/content/rssi.csv')

# Check for NaN or infinite values in your data
print(data.isna().sum())
print(data.isin([np.nan, np.inf, -np.inf]).sum())

# Remove rows with missing values
data.dropna(inplace=True)

# Replace infinite values with NaN
data.replace([np.inf, -np.inf], np.nan, inplace=True)

# One-hot encode categorical variables 'signal' and 'ap'
data = pd.get_dummies(data, columns=['signal', 'ap'])

# Split data into features (RSSI) and labels (x, y, z)
X = data.drop(columns=['x', 'y', 'z'])  # Features
y = data[['x', 'y', 'z']]  # Target variables

# Normalize the features using Min-Max scaling
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Reshape the input data to include time steps (1 in this case)
X = X.reshape(X.shape[0], 1, X.shape[1])

# Split the data into training (70%) and testing (30%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Define the LSTM model
model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(3))  # 3 output neurons for x, y, and z

# Create a learning rate schedule
def lr_schedule(epoch):
    if epoch < 30:
        return 0.001
    elif epoch < 60:
        return 0.0005
    else:
        return 0.0001

# Compile the model with a learning rate scheduler
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Set up early stopping to prevent overfitting
early_stopping = EarlyStopping(patience=20, restore_best_weights=True)

# Set up a model checkpoint to save the best model
checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)

# Learning rate scheduler callback
lr_scheduler = LearningRateScheduler(lr_schedule)

# Train the model
history = model.fit(
    X_train,
    y_train,
    epochs=200,  # Adjust as needed
    batch_size=32,  # Adjust as needed
    validation_data=(X_test, y_test),
    callbacks=[early_stopping, checkpoint, lr_scheduler],  # Apply early stopping and save the best model
    verbose=2,
)

# Evaluate the model on the testing data
y_pred = model.predict(X_test)

# Calculate the root mean squared error (RMSE)
rmse = sqrt(mean_squared_error(y_test, y_pred))
print(f'RMSE: {rmse}')

# Optionally, save the trained model
model.save('location_prediction_model.h5')

# Inverse transform predictions to the original scale
y_pred = scaler.inverse_transform(y_pred)
y_true = scaler.inverse_transform(y_test)

# Plot the results
plt.figure(figsize=(12, 6))
plt.plot(y_true, label='Actual', color='blue')
plt.plot(y_pred, label='Predicted', color='red')
plt.title('Actual vs. Predicted Location')
plt.xlabel('Sample')
plt.ylabel('Location (x, y, z)')
plt.legend()
plt.show()




ap          1
signal      1
sequence    1
x           1
y           1
z           1
dtype: int64
ap          1
signal      1
sequence    1
x           1
y           1
z           1
dtype: int64
Epoch 1/200
2625/2625 - 20s - loss: 75.3982 - val_loss: 57.5940 - lr: 0.0010 - 20s/epoch - 8ms/step
Epoch 2/200


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


2625/2625 - 16s - loss: 59.4019 - val_loss: 53.4413 - lr: 0.0010 - 16s/epoch - 6ms/step
Epoch 3/200
2625/2625 - 17s - loss: 57.0444 - val_loss: 52.8437 - lr: 0.0010 - 17s/epoch - 6ms/step
Epoch 4/200
2625/2625 - 16s - loss: 56.1494 - val_loss: 52.4172 - lr: 0.0010 - 16s/epoch - 6ms/step
Epoch 5/200
2625/2625 - 16s - loss: 55.4911 - val_loss: 52.2431 - lr: 0.0010 - 16s/epoch - 6ms/step
Epoch 6/200
2625/2625 - 15s - loss: 55.0762 - val_loss: 52.0925 - lr: 0.0010 - 15s/epoch - 6ms/step
Epoch 7/200
2625/2625 - 16s - loss: 54.7044 - val_loss: 51.8716 - lr: 0.0010 - 16s/epoch - 6ms/step
Epoch 8/200
2625/2625 - 15s - loss: 54.3845 - val_loss: 51.6935 - lr: 0.0010 - 15s/epoch - 6ms/step
Epoch 9/200
2625/2625 - 18s - loss: 54.2406 - val_loss: 51.6193 - lr: 0.0010 - 18s/epoch - 7ms/step
Epoch 10/200
2625/2625 - 16s - loss: 53.9579 - val_loss: 51.6859 - lr: 0.0010 - 16s/epoch - 6ms/step
Epoch 11/200
2625/2625 - 17s - loss: 53.7060 - val_loss: 51.2676 - lr: 0.0010 - 17s/epoch - 6ms/step
Epoch 12/2

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


ValueError: ignored

In [ ]:
# Make predictions
y_pred = model.predict(X_test_reshaped)

# Convert X_test_reshaped to float32 (if not already)
X_test_reshaped = X_test_reshaped.astype(np.float32)

# Inverse transform predictions to original scale
y_pred = scaler_y.inverse_transform(y_pred)

# Convert y_test (if needed) to float32 before inverse transforming
y_test = y_test.astype(np.float32)

# Inverse transform actual values to original scale
y_true = scaler_y.inverse_transform(y_test)

# Plot the results
plt.figure(figsize=(12, 6))
plt.plot(y_true, label='Actual', color='blue')
plt.plot(y_pred, label='Predicted', color='red')
plt.title('Actual vs. Predicted Location')
plt.xlabel('Sample')
plt.ylabel('Location (x, y, z)')
plt.legend()
plt.show()



ValueError: ignored